### Note
No need to calculate single gene essentiality, does not work when integrating more than one gene

In [1]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model

In [2]:
log_format = '%(asctime)s %(message)s' #%(clientip)-15s %(user)-8s
import logging
from coralme.builder.main import ListHandler
log = logging.getLogger()
logging.basicConfig(filename = "./ibdmdb/essentiality/essentiality.log", filemode = 'w', level = logging.WARNING, format = log_format)
logging.captureWarnings(True)

In [3]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [4]:
survivors = set(pandas.read_csv("survivors.txt",index_col=0,header=None).index.to_list())
done = set(i.split("/")[-1] for i in os.listdir("./ibdmdb/essentiality/") if ".csv" in i)

In [5]:
run_for = survivors

In [6]:
# Written originally by Rodrigo Santibanez for ME model
import copy
def essentiality(model,gene, info = None):
    msg = "Performing KO for {} in {}".format(gene,model.id)
    if info:
        msg = msg + " ({} out of {})".format(info[0],info[1])
    ListHandler.print_and_log(msg)
    test = copy.deepcopy(model)
    # test = model
    if isinstance(model,coralme.core.model.MEModel):
        for TU in test.transcription_data:
            data = test.transcription_data.get_by_id(TU.id)
            test.transcription_data.get_by_id(TU.id).RNA_products = data.RNA_products.difference([gene])
        for rxn in test.reactions.query('transcription_'):
            rxn.update()
        if test.feasibility({test.mu : 0.001}):
            return gene,False
        else:
            return gene,True
        
    # cobra model
    g = test.genes.get_by_id(gene)
    g.knock_out()
    sol = test.optimize()
    if sol.status != 'optimal' or sol.objective_value < 0.001:
        return gene, True
    else:
        return gene, False

In [7]:
import coralme
def run(org):
    dct = {}
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    ListHandler.print_and_log("Loaded {}".format(org))
    genes = model.all_genes
    for idx,gene in enumerate(genes):
        dct[gene.id] = essentiality(model,gene.id,info = (idx+1,len(genes)))[1]
    pandas.DataFrame.from_dict({"essential":dct}).to_csv("./ibdmdb/essentiality/{}.csv".format(org))

In [ ]:
NP = min([8,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)
    
for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

Building (8 threads):   0%|                                                                                                                                                                | 0/497 [00:00<?, ?it/s]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpoes48qc8.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp2p01u0dg.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmph9en2_9_.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp07wy9vu_.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp_g5uk0ik.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0